# AOI search via Jupyter notebook

In [2]:
import os
from modules import aoi, buttons
from ipywidgets import interact

## Draw a polygon on the map! 
- Select an area of interest (AOI) choosing a shape among the buttons on the left side
- Only one polygon per time is supported by this version
- *!!* Do not scroll world window and draw your AOI in a world copy. This will generate non valid coordinates for the query ([Issue #96](https://github.com/jupyter-widgets/ipyleaflet/issues/96))

In [3]:
m,dc = aoi.define_map()
def handle_draw(self, action, geo_json):
    print("Action: %s"%action)
    print(geo_json["geometry"])
    buttons.save_aoi(geo_json)

dc.on_draw(handle_draw)
m.add_control(dc)
display(m)

Map(center=[40.853294, 14.305573], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

Action: created
{'type': 'Polygon', 'coordinates': [[[137.763546, -43.216385], [137.763546, -9.994818], [162.912452, -9.994818], [162.912452, -43.216385], [137.763546, -43.216385]]]}

Selection saved


## Set some filters!
- Remember to submit each field (even if not set).

In [4]:
buttons._filters_()

DatePicker(value=None, description='Sensed from:')

DatePicker(value=None, description='Sensed to:')

Button(description='Submit sensing range', style=ButtonStyle())

Select(options={'All': ['-'], 'Sentinel-1': ['All S1', 'SLC', 'GRD', 'OCN', 'RAW'], 'Sentinel-2': ['All S2', '…

Button(description='Submit mission', style=ButtonStyle())

Sensing range from 2019-12-28 to 2019-12-31


Dropdown(description='Product type:', layout=Layout(width='30%'), options=('All S5P', 'L1B_IR_SIR', 'L1B_IR_UV…

Button(description='Submit product type', style=ButtonStyle())

In [5]:
polysel = buttons.read_aoi()
item = buttons.read_mp()
sensing = buttons.read_sen()
n = aoi.count(polysel,item,sensing)
df = aoi.search(n,polysel,item,sensing)

Searching for products...
https://catalogue.onda-dias.eu/dias-catalogue/Products/$count?$search=%22(%20(%20(%20platformName:Sentinel-5 Precursor%20AND%20productType:L2__HCHO__%20)%20)%20AND%20(%20(%20beginPosition:[2019-12-28T00:00:00.000Z%20TO%202019-12-31T23:59:59.999Z]%20AND%20endPosition:[2019-12-28T00:00:00.000Z%20TO%202019-12-31T23:59:59.999Z]%20)%20)%20)%20AND%20footprint:%22Intersects(POLYGON((137.763546%20-43.216385,137.763546%20-9.994818,162.912452%20-9.994818,162.912452%20-43.216385,137.763546%20-43.216385)))%22%22
12 products found


In [6]:
@interact
def _iFP_(product=[f for f in df.iloc[:,1]]): 
    keys = df.index[df['name'] == product].tolist()
    m = aoi.update_Map(keys,polysel,df)
    display(m)
    display(df.loc[keys])
    aoi.warning(df.loc[keys].iloc[:,6].any())
    print("Find item at: %s"%os.path.join(str(df.loc[keys].iloc[:,2].any()),str(df.loc[keys].iloc[:,1].any())))

interactive(children=(Dropdown(description='product', options=('S5P_OFFL_L2__HCHO___20191230T045347_20191230T0…

## Save your selections!

- Choose products you wish to collect and submit each of them. This will produce a dat set named *saved_query.csv* in the *output* folder. 
- *!!* Warning. Re-running this cell will erase the data set. Select and submit items in the current run.

In [6]:
sel = buttons.select_product(df)
sel.display_widgets() 

## Copy AOI dataframe in a destination folder
You may find useful to copy this dataframe into a destination folder. Specify the filename (saved_query.json or polygons.json) and the destination path. By default path is set to outputs folder.

In [11]:
file = "outputs/list.txt" # dataframe containing metadata 
dest = os.path.join(os.getcwd(),"S5P") # destination path here
aoi.copy2(os.path.join(os.getcwd(),file),dest) 

In [ ]:
file = "outputs/polygon.json" # selected polygon
dest = os.path.join(os.getcwd(),"S5P") # destination path here
aoi.copy2(os.path.join(os.getcwd(),file),dest) 